In [1]:
import pandas as pd

df1 = pd.read_csv('bench1_perfs.csv', sep=',', names=['duration', 'id', 'k', 'size2', 'nb_items', 'nb_ranks'])
df1 = df1.groupby(['id', 'k', 'size2', 'nb_items', 'nb_ranks'])['duration'].mean()
df1 = df1.rename('PIM').to_frame()

df2 = pd.read_csv('bench2_perfs.csv', sep=',', names=['duration', 'id', 'k', 'size2', 'nb_items'])
df2 = df2.groupby(['id', 'k', 'size2', 'nb_items'])['duration'].mean()
df2 = df2.rename('SyncCache').to_frame()

df = df1.join(df2)
df['PIM_Speedup'] = df['SyncCache'] / df['PIM']
print(df)

                                        PIM  SyncCache  PIM_Speedup
id     k size2 nb_items  nb_ranks                                  
init   8 30    10000000  6         0.151705   0.437144     2.881540
                         8         0.183400   0.437144     2.383550
               100000000 6         0.149947   0.517553     3.451577
                         8         0.196499   0.517553     2.633866
         31    10000000  6         0.141201   0.952125     6.743038
...                                     ...        ...          ...
weight 8 32    100000000 8         0.042688  13.709067   321.145226
         33    10000000  6         0.168461  27.853167   165.339353
                         8         0.084590  27.853167   329.273660
               100000000 6         0.169469  27.355533   161.418718
                         8         0.084656  27.355533   323.138992

[64 rows x 3 columns]


In [2]:
style = df.style.set_table_styles([
   {'selector': '',
    'props': [
        ('font-family', 'monospace'),
        ('border', '2px solid #e0e0e8'),
        ('text-align', 'left !important'),
        ('vertical-align', 'top'),
        ('color', '#343434'),
        ('font-size', '13px'),
    ]
    },
    {'selector': 'th',
    'props': [
        ('font-family', 'monospace'),
        ('border', '2px solid #e0e0e8'),
        ('text-align', 'left !important'),
        ('vertical-align', 'top'),
        ('padding', '5px'),
        ('color', '#343434'),
    ]
    }]
).format({'PIM_Speedup': '{:,.2f}'}).background_gradient(subset=['PIM_Speedup'], cmap='YlGn', vmin=0, vmax=10)
style